# Titanic Challenge

* This one can be in top 3% if you try a few times
* After figuring out that I have classified many who did not have a good chance to survive but actully survived as "Not Survived", I have added a random feature to make some of them become 0 to 1, and then I could enter the range inside top 3% 


## 1. Import libraries & data  데이터준비 및 모듈 임포트

In [ ]:
# They are for data manipulation/ 기본 데이터 정리 및 처리
import pandas as pd
import numpy as np

# For Visualization / 시각화
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn-whitegrid')
import missingno

# For preprocessing and ML algorithms / 전처리 및 머신 러닝 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

# Tunning and Evaluation / 모델 튜닝 및 평가
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn import model_selection

# Ignore warnings / 경고 제거 (Pandas often makes warnings)
import sys
import warnings

import warnings
warnings.filterwarnings('ignore')

### CSV to DF

import modules -> import data -> turning data into data frame (pandas) 

* 이어서 이를 데이터프레임에 임포트하여 데이터셋이 판다스 데이터프레임에 임포트 되도록 합니다.
* Import the data into the dataframe so the dataset is converted as a pandas dataframe.

In [ ]:
# This may be harder than expected for a firsttimer, but if the data was not delivered correctly from the copy, press "+ Add Data", load "Titanic Data" from "Competition Data", and click each file to check the path address.
# 이 것이 처음하는 사람에게 예상보다 어려울 수 있는데 복사한 것에서 데이터가 전달이 잘 안 되었다면 "+Add Data" 누르시고 'Competition Data'에서 "Titanic Data" 불러온 후 파일을 찍어서 경로 주소 확인해야 함 
test = pd.read_csv('../input/titanic/test.csv')
train = pd.read_csv('../input/titanic/train.csv')

# Now csv files, test and train, have become data frames.    

### Basic operations with dataframes

`head()` shows first five rows

In [ ]:
train.head()

`tail()` shows last five rows

In [ ]:
train.tail()

`describe()` shows basic statistics for columns. By default it shows information only about continuous features. You can see information about all features by setting `include='all'`

In [ ]:
train.describe(include='all')

`dtypes` shows types of all columns

In [ ]:
train.dtypes

`info()` is an advanced version of `dtypes` which shows not only types, but also the number of non-null values.

In [ ]:
train.info()

`columns` shows the list of all columns in dataframe

In [ ]:
train.columns

You can slice the list of columns like a usual python list object

In [ ]:
train.columns[3], train.columns[3:5]

You can select several rows of dataframe by indexing

In [ ]:
train[5:20]

`shape` shows number of rows and number of columns

In [ ]:
train.shape

## 2. 파일 분리 및 병합 (File Separations and Merges) 

* Secure the shape of ntrain and ntest. (Preparation for later splitting from the merged one)
* y_train is a known result.
* Separate the passenger ID of the test as it will be added to the final result later.
* Merge train and test to create a file called data. When you convert a string to a number, or group numbers by interval, it is easier for us to put them together to do it all at once.
* ntrain과 ntest의 shape을 확보해놓습니다. (병합 한 것을 나중에 다시 갈라 놓기 위한 준비)
* y_train은 알려진 결과 값이니 따로 모셔 놓고
* 테스트의 승객 아이디는 나중에 최종 결과에 넣을 것이기 때문에 따로 떼어 놓습니다.
* train과 test를 병합하여 data 란 파일을 만듭니다. 문자로 된 것을 숫자로 바꾼다든가.숫자를 인터발 별로 그룹화 한다든가 할 때 한꺼번에 하기 위해 합해 놓습니다.

In [ ]:
# 병합 준비
ntrain = train.shape[0]
ntest = test.shape[0]

# 아래는 따로 잘 모셔 둡니다.
y_train = train['Survived'].values
passId = test['PassengerId']

# 병함 파일 만들기
data = pd.concat((train, test))

# 데이터 행과 열의 크기는
print("data size is: {}".format(data.shape))

In [ ]:
train['Survived'].value_counts()

We can see that 342 people survived and 549 didn't survive

## 3. Check the files / 파일을 보겠습니다.

In [ ]:
missingno.matrix(data, figsize = (15,8))

This plot shows missing values in each column and in each row. Note that in the lower half of `Survived` column all the values are missing. This is because these rows are from test data.

In [ ]:
data.isnull().sum() #비어 있는 값들을 체크해 본다.

In [ ]:
data.Age.isnull().any()

* Names of columns / 열 이름을 보겠습니다.

In [ ]:
data.columns

## 4. Features 항목

#### 항목의 종류 There are many types of features.
* 범주형 항목 (Categorical Features)

범주형 변수는 둘 이상의 결과 요소가 있는 변수이며 해당 기능의 각 값을 범주별로 분류 할 수 있습니다. 예를 들어 성별은 두 가지 범주 (남성과 여성)의 범주 형 변수입니다. 이산형 변수(discrete variable) = 범주형 변수 (categorical variable) 의 하나로 명목 변수 norminal variable 라고도합니다.

Categorical variables are variables that have more than one result element, and each value of that function can be classified by category. For example, gender is a categorical variable in two categories (male and female). Discrete variable = It is one kind of categorical variable, and is also known as nominal variable .

* 데이터 셋에서 명목 항목 : Sex, Embark 이며 우리는 Name, Ticket 등을 이로 변환해야 할 것 같습니다. The nominal items in the data set are: Sex, Embark and we have to convert Name, Ticket, etc to numbers.

* Ordinal Variable :

순위 변수는 범주 형의 하나지만 그 차이점은 값 사이의 상대 순서(=서열) 또는 정렬이 가능하다는 것입니다.

Ordinal variables are one of the categorical types, but the difference is the relative order (= sequence) or sorting between the values.

데이터 셋에서 순위 항목 : PClass 이며 우리는 Cabin을 이 범주로 변환해서 사용해야 할 것 같습니다.

Ordinal variables in the data set: PClass and we think we should convert Cabin to this category.

* 연속형 항목 (Continuous Features):

서로 연속된 값을 가진 변수를 가진 항목이며 여기에서 우리는 연령을 대표적인 것으로 볼 수 있습니다.

This is an item with variables with continuous values, age is one good sample

Age, SipSp, Parch, Fare는 interval variable로 만들어 이에 적용해야 할 것 같습니다.

Age, SipSp, Parch, and Fare should be catgorized to the interval variable.

* Feature Information

          Variable        Definition                Key

          survival          Survival                    0 = No, 1 = Yes

          pclass          Ticket class                1 = 1st, 2 = 2nd, 3 = 3rd

          sex              Sex    

          Age              Age in years    

          sibsp              # of siblings / spouses aboard the Titanic    

          parch              # of parents / children aboard the Titanic    

          ticket          Ticket number    

          fare              Passenger fare    

          cabin              Cabin number    

          embarked          Port of Embarkation         C = Cherbourg, Q = Queenstown, S = Southampton

## 5. 데이터 탐구  Exploratory Data Analysis

* train파일 순서대로 데이터 파일의 열들을 봅니다. 
* Let's view the columns of the data file in the order in the train file.

![](https://1.bp.blogspot.com/-rBTabaGeOTo/XicYRmv9s7I/AAAAAAAAKts/WQDUpGJbv20xbAO8vfnOkqtbCHyme3zNQCLcBGAsYHQ/s640/grey%2Barea.png)

In [ ]:
train.head()

* 파일 각 열의 상관 관계를 보겠습니다.

Let's take a look at the correlation of each column in the file.

Co-relation 매트릭스는 seaborn에서 변수 간 상관 계수를 보여주는 표입니다. 표의 각 셀은 두 변수 간의 상관 관계를 보여줍니다. 상관 매트릭스는 고급 분석에 대한 입력 및 고급 분석에 대한 진단으로 데이터를 요약하는 데 사용됩니다. 참고: https://seaborn.pydata.org/examples/many_pairwise_correlations.html

Co-relation matrix is a table showing the correlation coefficient between variables in seaborn. Each cell in the table shows a correlation between two variables. Correlation matrices are used to summarize data as input to advanced analysis and as diagnostics for advanced analysis. Note: https://seaborn.pydata.org/examples/many_pairwise_correlations.html

아래 마스크 셋업은 0로 행렬을 상관 행렬과 같은 모양으로 만든 후 여기에 불리안 값을 넣고 이를 다시 True만 만듭니다.

The mask setup below makes a matrix that looks like a correlation matrix with zeros, then puts a Boolean value into it and makes it true.

triu 는 우측 상단 삼각행렬을 의미

triu means upper right triangle

annot= True는 각 셀에 숫자를 표시하라는 것이고, False는 하지 말라는 것이구요

annot = True means to display a number in each cell, False means oppposite. https://seaborn.pydata.org/generated/seaborn.heatmap.html

이어서 이를 heatmap으로 런칭합니다.

Then launch it as a heatmap.

In [ ]:
# Co-relation 매트릭스
corr = data.corr()
# 마스크 셋업
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
# 그래프 셋업
plt.figure(figsize=(14, 8))
# 그래프 타이틀
plt.title('Overall Correlation of Titanic Features', fontsize=18)
#  Co-relation 매트릭스 런칭
sns.heatmap(corr, mask=mask, annot=False,cmap='RdYlGn', linewidths=0.2, annot_kws={'size':20})
plt.show()

* "Surived" 분석

한 열씩 검토해 보겠습니다.

We will look at the columns one by one

Survived - Key: (0 - Not Survived, 1- Survived)

Survived는 수자로 값을 주지만 Categorical Variable인 셈입니다.

죽던지 살던지 둘 중 하나의 값을 줍니다.

countplot을 그려 봅니다.

사이즈는 가로 10인치 세로 2인치

생존 여부 0과 1의 숫자를 세어 본 후 그림을 그리도록 명령을 하는 것입니다.

pyplot(plt)의 figure라는 메소드를 써서 그림판의 크기를 정하고, seaborn의 카운트플롯을 그리라는 것입니다.

"Survived" gives a number, but it's a categorical variable.

Either Survived or Not Survived

Draw a countplot.

Size is 10 inches wide by 2 inches long

Survival counts

Use pyplot (plt) figure method to size the paint and draw a seaborn count plot.

In [ ]:
fig = plt.figure(figsize=(10,2))
sns.countplot(y='Survived', data=train)
print(train.Survived.value_counts())

* 불행히도 사망자가 훨씬 많아 보입니다.
* 전체 사망자 비율을 좀 보겠습니다.
 
* Unfortunately, there are more deaths than "survived".
 
* Let's take a look at the overall rate.
* 파이그래프랑 카운트 플롯을 서브플롯으로 그립니다.
* 행은 하나 열은 2개의 서브 플롯입니다. 사이즈는 가로 15인치 세로 6인치
* 'Survived'의 값을 카운트해서 파이플롯을 만듭니다.
* explode는 폭발하는 것이니까 1이면 튀어 나가는 것인데 0을 주면 분리만 되고 돌출은 되지 않습니다. 이어서 0, 1인 것은 첫 번째 것은 아니고 두번 째 것은 분리된다는 의미로 생각하시면 됩니다.
* autopercent는 1.1이 표현하는 부분은 소수점 한 자리까지 보여 주라는 의미입니다. 뒤에 점 이하가 4면 둘 다 소수점 4자리수 까지 보여 줍니다.
* ax[0]은 첫번째 칸입니다.
* set_title 메소드는 서브 플롯의 제목을 보여 줍니다.
 
* Draw a pie chart and count plot in the subplot parts.
 
* Two rows and one column. Size : 15 inches wide by 6 inches high
* Create a pieplot by counting the value of 'Survived'.
* Explode means "protrusion", so if it's 1, it's popping out. 0 and 1 means that the second one is poping out.
* Autopercent means 1.1 shows up to one decimal place. If there is 4 after the dot, both show up to four decimal places.
* ax [0] is the first cell.
* The set_title method displays the title of the subplot.

In [ ]:
f,ax=plt.subplots(1, 2, figsize=(15, 6))
train['Survived'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Survived')
ax[0].set_ylabel('')
sns.countplot('Survived',data=train, ax=ax[1])
ax[1].set_title('Survived')
plt.show()

* 위의 것을 아래와 같이 함수로 만들겠습니다. (물론 자주 쓰이지는 않겠지만 연습이니)
* Let's make the above as a function. (it's not going to be used often, but it's a practice for making a function)

In [ ]:
def piecount(col):
    f, ax = plt.subplots(1, 2, figsize=(15, 6))
    train[col].value_counts().plot.pie(explode=[0.1 for i in range(train[col].nunique())], autopct='%1.1f%%', ax=ax[0], shadow=True)
    ax[0].set_title(col)
    ax[0].set_ylabel('')
    sns.countplot(col, data=train, ax=ax[1])
    ax[1].set_title(col)
    plt.show()

piecount('Survived')

This function works for features with any number of categories

In [ ]:
piecount("Pclass")

* "Pclass" 분석

* Pclass는 값이 숫자이나 서열이 정해진 Ordinal Feature이다.
* Pclass is an "Ordinal Feature" whose values are numerical but sequenced.
* Key:1 = 1st, 2= 2nd, 3 = 3rd
* 
* 각 클래스 당 생존자를 보겠습니다.
* * Let's look at the survivors for each class.

In [ ]:
train.groupby(['Pclass','Survived'])['Survived'].count()

In [ ]:
pd.crosstab(train.Pclass, train.Survived, margins=True).style.background_gradient(cmap='summer_r')

* 1등급 객실의 사람들은 생존자가 더 많고, 2등급은 생존자에 비해 사망자가 조금 더 많으나, 3등급은 사망자가 3배 이상 많다는 것을 알 수 있습니다.
* The 1st class cabin has more survivors, the 2nd class has a few more deaths than the survivors, but the 3rd class has three times more deaths.

In [ ]:
f, ax = plt.subplots(1, 2, figsize=(12, 6))
train[['Pclass','Survived']].groupby(['Pclass']).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived per Pcalss')
sns.countplot('Pclass', hue='Survived', data=train, ax=ax[1])
ax[1].set_title('Pcalss Survived vs Not Survived')
plt.show()

* 위에 만든 함수를 한 번 써 먹어 볼까요?
* Let's use a function we made above.

In [ ]:
piecount("Pclass")

* %는 3등칸이 반이 넘으나 위의 그래프에서 생존자는 1등석이 가장 많다는 것을 알 수 있습니다.
* 각 클래스 당 생존률을 볼까요?

### "Name" 분석
* 이름은 거의 모두 다를 가능성이 큽니다. Family Name, First Name, Middle Name and even Dr. Capt, master and so on 모두 감안하면...
* 분류를 한 번 해 봅니다.
* 리스트를 한 번 주 욱 보겠습니다.
* The names are most likely different...all of them . Family Name, First Name, Middle Name and even Dr. Given Capt, Master and so on ...
* Try to classify them.
* Let's look at the list once.

In [ ]:
data.Name.value_counts()

* 이름은 언뜻 보아서 감이 안 옵니다. 중간에 있는 Mr. 같은 호칭을 볼까요.
* ['Initial']이란 열을 새로 만들어서 여기에 Name에서 추출한 Regular Expression을 넣습니다.
* 아래에서 str.extract('([A-Za-z]+).')부분은 str에서 대문자 A~Z, 소문자 a~z 중에 . 명령을 통해 .으로 끝나는 부분을 추출해 내는 것입니다.
* ('^([A-Za-z]+)')으로 하면 처음에 나오는 문자 덩어리가 될 것이고 +를 빼면 첫 스펠링 한캐릭터만 추출합니다.
 
* The name doesn't seem to have any meaning at first glance.
 
* Create a new column called ['Initial'] and put the regular expressions extracted from Name.
* The str.extract ('([A-Za-z] +) .') Part of the lower part of the str is the . Is to extract the part that ends with.
* ('^ ([A-Za-z] +)') will be the first chunk of characters, and minus + will extract only the first spelled character.

* 안전을 위해 카피를 하나 만들어서 새로운 항목을 만들어 봅니다.
* Just in case, let's make a copy of the df and make new features on it.

In [ ]:
temp = data.copy()
temp['Initial'] = 0
temp['Initial'] = data.Name.str.extract('([A-Za-z0-9]+)\.')

We can see that several titles are common, but most of them are rare.

In [ ]:
temp['Initial'].value_counts()

이를 성별로 봅니다.
View it by gender.

In [ ]:
pd.crosstab(temp.Initial, temp.Sex).T.style.background_gradient(cmap='summer_r')

* 생존률로 봅니다.
* by survival percentage

In [ ]:
def survpct(col):
    return temp.groupby(col)['Survived'].mean()

survpct('Initial')

* 생존 숫자로 봅니다.
* by number

* test 에 있는 Dona의 나이를 보고 어디에 넣을지 보겠습니다.
* Ms. 는 현대처럼 Miss + Mrs를 합친 말이 아니라 당시에는 귀족미망인을 의미하는 것이 었습니다. Mlle나 Mme등도 마드모아젤과 마담의 줄인말일 경우일 것입니다. 귀족 여성들로 보아야겠죠.
 
* Let's look at Dona's age and decide to where to classify her.
  
* Ms. did not mean Miss + Mrs combined like these days, at the time it meant widow. Also Mlle, & Mme may be short for Mademoiselle and Madame. I should see them as women in a higher class.

In [ ]:
temp['LastName'] = data.Name.str.extract('([A-Za-z]+)')

In [ ]:
pd.crosstab(temp.LastName, temp.Survived).T.style.background_gradient(cmap='summer_r')

* 이제 우리는 Initial에서 Mr.등의 호칭을 뽑아내었고, 성을 뽑아내었습니다.
* Now we have extracted salutations in 'Initial' and last names in 'LastName'

* 머신이 알파벳보다는 숫자를 좋아 하므로 숫자로 바꿉니다.
* Since our machine prefers numbers over alphabets, we change them to numbers.

* 아, 그러기 전에 Dona를 처리해야지요.
* But we have to take care of Dona first.

In [ ]:
temp.loc[temp['Initial'] == 'Dona']

* 나이로 추측해서 Mrs.로 넣습니다.
* Let me guess she is Mrs. based on her age.

In [ ]:
temp.loc[temp['Initial'] == 'Dona', 'Initial'] = 'Mrs'

In [ ]:
pd.crosstab(temp.Initial, temp.Survived).T.style.background_gradient(cmap='summer_r')

* Last name 은 전부 숫자로 바꿉니다.
* Let's change the Last Names to numbers.

In [ ]:
temp['NumName'] = temp['LastName'].factorize()[0]

In [ ]:
pd.crosstab(temp.NumName, temp.Survived).T.style.background_gradient(cmap='summer_r')

In [ ]:
temp.loc[temp['LastName'] == 'Ali']

* 보시다시피 같은 Last name에 같은 번호가 쓰여졌다.
* As you see NumnName is the same as their Last Names are the same.
* The part at the end: [0] means you are only taking the labels, throwing away the uniques that map back to your input.
* 끝에 [0]은 라벨만 보고 번호를 붙이는 것으로 정말 unique한 것이란 것은 안 본다는 것입니다.


* We have turned 'Initial' & 'NumName' representing "Name" and "Last Name" into numbers. Let's move on.
* 자 이제 이름을 의미하는 중요한 요소 두 개를 숫자로 바꾸었으니 다음으로 갑니다.

### "Sex" 분석

* 함수를 만들어서 train파일을 보지요
* Let's see the graph Survived / Sex

In [ ]:
train[['Sex','Survived']].groupby(['Sex']).mean()

In [ ]:
def bag(col, target, title, title1):
    f,ax=plt.subplots(1,2,figsize=(12,5))
    train.groupby([col])[target].mean().plot(kind='bar', ax=ax[0])
    ax[0].set_title(title)
    sns.countplot(col, hue=target, data=train, ax=ax[1])
    ax[1].set_title(title1)
    plt.show()

bag('Sex','Survived','Survived per Sex','Sex Survived vs Not Survived')

* 배에 있던 남자의 수는 여자의 수보다 훨씬 많습니다. 여전히 생존 여성 수는 남성 수의 거의 두 배입니다. 선박 여성의 생존율은 약 75 % 인 반면 남성의 생존율은 약 18-19 %입니다.
* The number of men on the ship is much more than the number of women. Still, the number of surviving women is almost twice that of men. The survival rate of ship women is about 75%, while the survival rate of men is about 18-19%.
* 이 것은 남성/여성을 1,2로 나누면 될 것 같은 뻔해 보이는 것이지만 좀 더 새분화하면 좋아 보입니다.
* 예를 들어 아기들은 아기이지, 남자인지 여자인지 구명보트 태울 때 안 물어 볼 것이기 때문입니다.
* 오히려 (불행하게도) 귀족 아기인지 서민의 아기인지는 행과불행을 가를 수 있습니다 ㅠㅠ
* 생존 Pclass별로 성별을 봅니다.
* This seems obvious to divide males / females by 1,2, but it looks good if you break it down a bit.
* For example, babies are just babies and would not be matter whether it is boy or girl.
* (Unfortunately) whether you are a baby in a higher social class or one from a humble family, it might have mattered.
* View gender by survival Pclass.

In [ ]:
pd.crosstab([train.Sex, train.Survived],train.Pclass,margins=True).style.background_gradient(cmap='summer_r')

* 사회는 불공평 했으나 최소한 남자들의 신사도는 있었다고 할 수 있을 것 같습니다.
* The society was unfair but gentlemenship worked there.

### Age 분석
 
* Age는 Continuous한 값입니다.
* 빈칸이 많아서 빈칸처리가 결정적인 역할을 할 것 같습니다.
 
* Age의 최대, 최소, 중간을 보겠습니다.
 
* Age is a continuous value.
 
* Because there are a lot of blanks, blank processing seems to play a decisive role.
 
* Let's take a look at the maximum, minimum, and middle of Age.

In [ ]:
print('Oldest Passenger was', data['Age'].max(), 'Years')
print('Youngest Passenger was', data['Age'].min(), 'Years')
print('Average Age on the ship was', int(data['Age'].mean()), 'Years')

In [ ]:
sns.swarmplot(x=train['Survived'], y=train['Age'])
plt.xlabel("Survived")
plt.ylabel("Age")
plt.show()

In [ ]:
f, ax = plt.subplots(1,2,figsize=(18,8))
sns.violinplot("Pclass", "Age", hue="Survived", data=train, split=True, ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0, 110, 10))
sns.violinplot("Sex","Age", hue="Survived", data=train, split=True, ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0, 110, 10))
plt.show()

#### 관찰 :

1) Pclass에 따라 어린이 수가 증가하고 10 세 미만의 어린이 (즉, 어린이)의 생존율은 Pclass에 상관없이 양호해 보입니다.

2) Pclass1에서 20-50세의 Passeneger의 생존 가능성은 높고 여성에게는 더 좋습니다.

3) 남성의 경우 생존 확률은 나이가 증가함에 따라 감소합니다.

#### observation :

1) The number of children increases with Pclass, and the survival rate for children under 10 years old (i.e. children) looks good regardless of Pclass.

2) Passeneger, 20-50 years old, in Pclass1 has a higher chance of survival and is better for women.

3) In men, the probability of survival decreases with age.

우선 age의 빈칸 부터 해결 합니다.

First, resolve the issue of null values of age.

앞에서 살펴본 것처럼 Age 항목에는 177 null 값이 있습니다. 이러한 NaN 값을 대체하기 위해 데이터 집합의 평균 수명을 지정할 수 있습니다.

그러나 문제는 평균 연령이 29 세를 4세 아이에게 할당 할 수 없습니다. 승객이 어떤 연령대에 있는지 알 수있는 방법이 있을까요? 이름에서 힌트를 찾아 봅니다.

As we saw earlier, the Age item has a 177 null values. To replace these NaN values, you can specify the average age of the dataset.

But the problem is that the average age is 29 years old can not be assigned this to a kid. Is there any other way ? Look for some hints in their names.

* 그리고 Initial 별 평균 연령을 보고 Age에 적용 시키는 것이 좋을 것 같습니다.
* And it is better to see the average age by Initial and apply it to Age.

In [ ]:
temp.groupby('Initial').agg({'Age': ['mean', 'count']}) #이니셜 별 평균 연령 체크

In [ ]:
# Assining NaN Age items with mean value of Initials
temp = temp.reset_index(drop=True)

temp['Age'] = temp.groupby('Initial')['Age'].apply(lambda x: x.fillna(x.mean()))

temp[31:50]

* 이제 Initial을 좀 정리합니다.

In [ ]:
temp['Initial'].replace(['Capt', 'Col', 'Countess', 'Don', 'Dona' , 'Dr', 'Jonkheer', 'Lady', 'Major', 'Master',  'Miss'  ,'Mlle', 'Mme', 'Mr', 'Mrs', 'Ms', 'Rev', 'Sir'], ['Sacrificed', 'Respected', 'Nobles', 'Mr', 'Mrs', 'Respected', 'Mr', 'Nobles', 'Respected', 'Kids', 'Miss', 'Nobles', 'Nobles', 'Mr', 'Mrs', 'Nobles', 'Sacrificed', 'Nobles'],inplace=True)
temp['Initial'].replace(['Kids', 'Miss', 'Mr', 'Mrs', 'Nobles', 'Respected', 'Sacrificed'], [4, 4, 2, 5, 6, 3, 1], inplace=True)

In [ ]:
temp['Age_Range'] = pd.qcut(temp['Age'], 10)

In [ ]:
survpct('Age_Range')

In [ ]:
temp['Agroup'] = 0

temp.loc[temp['Age'] < 1.0, 'Agroup'] = 1
temp.loc[(temp['Age'] >=1.0) & (temp['Age'] <= 3.0), 'Agroup'] = 2
temp.loc[(temp['Age'] > 3.0) & (temp['Age'] < 11.0), 'Agroup'] = 7
temp.loc[(temp['Age'] >= 11.0) & (temp['Age'] < 15.0), 'Agroup'] = 13
temp.loc[(temp['Age'] >= 15.0) & (temp['Age'] < 18.0), 'Agroup'] = 16
temp.loc[(temp['Age'] >= 18.0) & (temp['Age'] <=  20.0), 'Agroup'] = 18
temp.loc[(temp['Age'] > 20.0) & (temp['Age'] <= 22.0), 'Agroup'] = 21
temp.loc[(temp['Age'] > 22.0) & (temp['Age'] <= 26.0), 'Agroup'] = 24
temp.loc[(temp['Age'] > 26.0) & (temp['Age'] <= 30.0), 'Agroup'] = 28
temp.loc[(temp['Age'] > 30.0) & (temp['Age'] <= 32.0), 'Agroup'] = 31
temp.loc[(temp['Age'] > 32.0) & (temp['Age'] <= 34.0), 'Agroup'] = 33
temp.loc[(temp['Age'] > 34.0) & (temp['Age'] <= 38.0), 'Agroup'] = 36
temp.loc[(temp['Age'] > 38.0) & (temp['Age'] <= 52.0), 'Agroup'] = 45
temp.loc[(temp['Age'] > 52.0) & (temp['Age'] <= 75.0), 'Agroup'] = 60
temp.loc[temp['Age'] > 75.0, 'Agroup'] = 78


In [ ]:
temp.head()

* Age는 그룹화 시키면 좋으나 학습을 위해서 그냥 놓아두고, 그룹화 연습은 Fare로 하겠습니다.
* Age should be grouped, but we are focusing on practices not on competition itself, we will just let it be as is and group "fare" later on. 

* Let's combine Pclass & Sex and make one coulmn called Gclass = Gender class

In [ ]:
temp['Gclass']=0
temp.loc[((temp['Sex'] == 'male') & (temp['Pclass'] == 1)), 'Gclass'] = 1
temp.loc[((temp['Sex'] == 'male') & (temp['Pclass'] == 2)), 'Gclass'] = 2
temp.loc[((temp['Sex'] == 'male') & (temp['Pclass'] == 3)), 'Gclass'] = 2
temp.loc[((temp['Sex'] == 'female') & (temp['Pclass'] == 1)), 'Gclass'] = 3
temp.loc[((temp['Sex'] == 'female') & (temp['Pclass'] == 2)), 'Gclass'] = 4
temp.loc[((temp['Sex'] == 'female') & (temp['Pclass'] == 3)), 'Gclass'] = 5
temp.loc[(temp['Age'] < 1), 'Gclass'] = 6

In [ ]:
survpct('Gclass')

#### Family or Alone?


In [ ]:
# Fill in missing Fare value by overall Fare mean
temp['Fare'].fillna(temp['Fare'].mean(), inplace=True)

# Setting coin flip (e.g. random chance of surviving)
default_survival_chance = 0.5
temp['Family_Survival'] = default_survival_chance

# Grouping data by last name and fare - looking for families
for grp, grp_df in temp[['Survived','Name', 'LastName', 'Fare', 'Ticket', 'PassengerId',
                           'SibSp', 'Parch', 'Age', 'Cabin']].groupby(['LastName', 'Fare']):
    
    # If not equal to 1, a family is found 
    # Then work out survival chance depending on whether or not that family member survived
    if (len(grp_df) != 1):
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Survived'].max()
            smin = grp_df.drop(ind)['Survived'].min()
            passID = row['PassengerId']
            if (smax == 1.0):
                temp.loc[temp['PassengerId'] == passID, 'Family_Survival'] = 1
            elif (smin == 0.0):
                temp.loc[temp['PassengerId'] == passID, 'Family_Survival'] = 0

# Print the headline
print("Number of passengers with family survival information:", 
      temp.loc[temp['Family_Survival']!=0.5].shape[0])

In [ ]:
# If not equal to 1, a group member is found
# Then work out survival chance depending on whether or not that group member survived
for _, grp_df in temp.groupby('Ticket'):
    if (len(grp_df) != 1):
        for ind, row in grp_df.iterrows():
            if (row['Family_Survival'] == 0) | (row['Family_Survival']== 0.5):
                smax = grp_df.drop(ind)['Survived'].max()
                smin = grp_df.drop(ind)['Survived'].min()
                passID = row['PassengerId']
                if (smax == 1.0):
                    temp.loc[temp['PassengerId'] == passID, 'Family_Survival'] = 1
                elif (smin==0.0):
                    temp.loc[temp['PassengerId'] == passID, 'Family_Survival'] = 0

# Print the headline
print("Number of passenger with family/group survival information: " 
      +str(temp[temp['Family_Survival']!=0.5].shape[0]))

In [ ]:
bag('Parch', 'Survived', 'Survived per Parch', 'Parch Survived vs Not Survived')

In [ ]:
pd.crosstab([temp.Family_Survival, temp.Survived], temp.Pclass, margins=True).style.background_gradient(cmap='summer_r')

In [ ]:
# Creating two features of relatives and not alone
temp['Family Size'] = temp['SibSp'] + temp['Parch']

In [ ]:
survpct('Family Size')

#### "Ticket"분석
* Ticket의 형태를 보겠습니다.
* Let's look at the numbers of the ticket.

In [ ]:
temp.Ticket.head()

* 도무지 감이 안 잡히는 배열입니다.
* 빈칸이 없는지 보겠습니다.
 
* It is an array that may not make any sense.
* Let's see if there are any blanks.

In [ ]:
temp.Ticket.isnull().any()

* 티켓에서 영문있는 것과 숫자만 있는 것을 따봅니다.
* See the tickets with letters or numbers only 

In [ ]:
temp['Initick'] = temp.Ticket.str.extract('^([A-Za-z]+)')

temp = temp.reset_index(drop=True)  # to avoid `ValueError: cannot reindex from a duplicate axis`

temp.loc[temp.Initick.isnull(), 'Initick'] = temp['Ticket']

temp.head()

In [ ]:
temp['NumTicket'] = temp['Initick'].factorize()[0]

In [ ]:
temp.head(n=15)

In [ ]:
temp.groupby('NumTicket')['Survived'].mean().to_frame().plot(kind='hist')
plt.title('Distribution of survival rate for different tickets');

### "Fare" 분석

In [ ]:
print('Highest Fare was:', temp['Fare'].max())
print('Lowest Fare was:', temp['Fare'].min())
print('Average Fare was:', temp['Fare'].mean())

In [ ]:
f,ax=plt.subplots(1, 3, figsize=(20, 6))
sns.distplot(train[train['Pclass'] == 1].Fare,ax=ax[0])
ax[0].set_title('Fares in Pclass 1')
sns.distplot(train[train['Pclass'] == 2].Fare,ax=ax[1])
ax[1].set_title('Fares in Pclass 2')
sns.distplot(train[train['Pclass'] == 3].Fare,ax=ax[2])
ax[2].set_title('Fares in Pclass 3')
plt.show()

* Pclass1의 승객 요금에는 큰 분포가있는 것으로 보이며 불연속 값으로 변환 할 수 있습니다.
* The passenger fare for Pclass1 seems to have a large distribution and can be converted to discrete values.

* Fare를 그룹으로 나누어 놓겠습니다.
* qcut을 활용하면 원하는 조각으로 데이터를 나누어 줍니다.
 
* I will divide the Fare into groups.
* Use qcut to divide the data into the desired pieces.

In [ ]:
def groupmean(a,b):
    return temp.groupby([a])[b].mean().to_frame().style.background_gradient(cmap='summer_r')

temp['Fare_Range'] = pd.qcut(train['Fare'], 10)
groupmean('Fare_Range', 'Fare')

* Fare를 그룹화 시킵니다. Fgroup이라고 이름 짓겠습니다.
* Group Fare with the name of Fgroup

0 and below -> 0

7.125 and below-> 5.0

7.9 and below-> 7.5

8.03 or less-> 8.0

Less than 10.5-> 9.5

Less than 23-> 16.0

27.8 and below-> 25.5

51 and below-> 38

73.5 and below-> 62

Over 73.5-> 100

In [ ]:
temp['Fgroup'] = 0

temp.loc[temp['Fare'] <= 0,'Fgroup'] = 0
temp.loc[(temp['Fare'] > 0) & (temp['Fare'] <= 7.125), 'Fgroup'] = 1
temp.loc[(temp['Fare'] > 7.125) & (temp['Fare'] <= 7.9), 'Fgroup'] = 2
temp.loc[(temp['Fare'] > 7.9) & (temp['Fare'] <= 8.03), 'Fgroup'] = 3
temp.loc[(temp['Fare'] > 8.03) & (temp['Fare'] < 10.5), 'Fgroup'] = 4
temp.loc[(temp['Fare'] >= 10.5) & (temp['Fare'] < 23.0), 'Fgroup'] = 5
temp.loc[(temp['Fare'] >= 23.0) & (temp['Fare'] <= 27.8), 'Fgroup'] = 6
temp.loc[(temp['Fare'] > 27.8) & (temp['Fare'] <= 51.0), 'Fgroup'] = 7
temp.loc[(temp['Fare'] > 51.0) & (temp['Fare'] <= 73.5), 'Fgroup'] = 8
temp.loc[temp['Fare'] > 73.5, 'Fgroup'] = 9

temp.head()

### "Cabin" 분석
* cabin 의 위치에 따라 달라지는 것이 있는지 보겠습니다.
* Let's see if there is anything that depends on the location of the cabin.

In [ ]:
temp.Cabin.value_counts().head(10)

In [ ]:
temp.Cabin.isnull().sum()

* 빈칸이 무척 많습니다.
* Lots of null values
* Cabin에 비어 있는 것이 많아 이를 다른 분류로 일단 잡고 기존 것은 이니셜로 분류합니다.
* 빈 것은 X로 구분하려는데 이 또한 1,2,3 Pclass와 연동될 것 같으니 비어있고 1등급은 X, 2등급은 Y, 3등급은 Z로 하겠습니다.
 
* There are so many empty bins in the cabin,
* Existing ones are classified as initials.
* I want to classify the nulls as X, but this is also reclassified by Pclasses, so there will be X, Y and Z

In [ ]:
temp['Inicab'] = 0
temp['Inicab'] = temp['Cabin'].str.extract('^([A-Za-z]+)')
temp.loc[((temp['Cabin'].isnull()) & (temp['Pclass'].values == 1)), 'Inicab'] = 'X'
temp.loc[((temp['Cabin'].isnull()) & (temp['Pclass'].values == 2)), 'Inicab'] = 'Y'
temp.loc[((temp['Cabin'].isnull()) & (temp['Pclass'].values == 3)), 'Inicab'] = 'Z'
    
temp.head()

In [ ]:
temp['Inicab'] = temp['Inicab'].factorize()[0]
    
temp[11:20]

#### "Embarked" 분석

In [ ]:
pd.crosstab([temp.Embarked, temp.Pclass], [temp.Sex, temp.Survived], margins=True).style.background_gradient(cmap='summer_r')

* Survival rates per Embarked ports
* 승선 장소 별로 생존 확률

In [ ]:
sns.factorplot('Embarked', 'Survived', data=temp)
fig = plt.gcf()
fig.set_size_inches(5, 3)
plt.show()

In [ ]:
f,ax=plt.subplots(2,2,figsize=(20,15))
sns.countplot('Embarked', data=temp, ax=ax[0,0])
ax[0,0].set_title('No. Of Passengers Boarded')
sns.countplot('Embarked', hue='Sex', data=temp, ax=ax[0,1])
ax[0,1].set_title('Male-Female Split for Embarked')
sns.countplot('Embarked', hue='Survived', data=temp, ax=ax[1,0])
ax[1,0].set_title('Embarked vs Survived')
sns.countplot('Embarked', hue='Pclass', data=temp, ax=ax[1,1])
ax[1,1].set_title('Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2,hspace=0.5)
plt.show()

1) 포트 C의 생존 가능성은 0.55 정도이며 S는 가장 낮습니다.S에서 탑승 최대. 대다수는 Pclass3

2) C의 승객들은 많은 비율이 살아남았습니다. 그 이유는 Pclass1 및 Pclass2 승객이 많아서 일 것입니다

3) Embark S는 대부분의 부자들이 탑승한 항구지만 생존 가능성은 낮습니다. Pclass3의 승객도 많았습니다.

4) 포트 Q는 승객의 거의 95 %가 Pclass3

observation :

1) Maximum boarding in S. The majority is Pclass3

2) Passengers of C survived a large proportion. The reason would be due to the large number of Pclass1 and Pclass2 passengers

3) Embark S is the port where most rich people board, but it is unlikely to survive. Lots of passengers in Pclass3.

4) Port Q has almost 95% of passengers Pclass3

* 빈칸이 두개 있는데 보겠습니다.
* Let us see 2 null values

In [ ]:
temp.loc[(temp.Embarked.isnull())]

* 두 사람의 티켓 번호가 같습니다.
* 혹시 같은 티켓 번호가 있는 다른 사람이 있는지 봅니다.
 
* Two people have the same ticket number.
* See if anyone else has the same ticket number.

In [ ]:
temp.loc[(temp.Ticket == '113572')]

* 가장 비슷한 번호를 찾아 보겠습니다.
* Let's find the similar numbers.

In [ ]:
temp.sort_values(['Ticket'], ascending = True)[55:70]

* 앞 뒤로 모두 S이고 Pclass도 모두 1인 것으로 봐서 S일 가능성이 큽니다.
* It is most likely that it is S because both front and back are S and Pclass is all 1.

In [ ]:
temp.loc[(temp.Embarked.isnull()), 'Embarked'] = 'S'

In [ ]:
temp.loc[(temp.Embarked.isnull())]

In [ ]:
temp['Embarked'] = temp['Embarked'].factorize()[0]
    
temp[11:20]

## 6. Feature Engineering

### 6.1 Turning string to numbers

* Already done above

### 6.2 Adding Features

* Some done above

* 그래도 몇 개 만들어 볼까요? But why don't we try making some more

* Priority - (1) Nobles (2) Women in Pclass 1 (3) Babies under 1 (4) Kids under 17 in Pclass 1 & 2  (5) Women in Pclass 3
* FH - Female Higher Survival Group
* MH - Male Higher Survival Group
* FL - Female Lower Surival Group
* ML - Male Lower Survival Group

* And I will keep making new ones until all the data are either over 80% or le

### Priority - (1) Nobles (2) Women in Pclass 1 (3) Babies under 1 (4) Kids under 17 in Pclass 1 & 2  (5) Women in Pclass 2 (6) Higher Fare

In [ ]:
temp['Priority'] = 0
temp.loc[(temp['Initial'] == 6), 'Priority'] = 1
temp.loc[(temp['Gclass'] == 3), 'Priority'] = 2
temp.loc[(temp['Gclass'] == 6), 'Priority'] = 3
temp.loc[(temp['Pclass'] == 1) & (temp['Age'] <= 17), 'Priority'] = 4
temp.loc[(temp['Pclass'] == 2) & (temp['Age'] <= 17), 'Priority'] = 5
temp.loc[(temp['Pclass'] == 2) & (temp['Sex'] == 2), 'Priority'] = 6
temp.loc[(temp['Fgroup'] == 9), 'Priority'] = 7

survpct('Priority')

In [ ]:
temp['FH'] = 0
temp.loc[(temp['Gclass'] == 1), 'FH'] = 0
temp.loc[(temp['Gclass'] == 2), 'FH'] = 0
temp.loc[(temp['Gclass'] == 3), 'FH'] = 1
temp.loc[(temp['Gclass'] == 4) & (temp['Family Size'] == 2), 'FH'] = 2
temp.loc[(temp['Gclass'] == 4) & (temp['Family Size'] == 3), 'FH'] = 3
temp.loc[(temp['Gclass'] == 4) & (temp['Family Size'] == 4), 'FH'] = 4
temp.loc[(temp['Gclass'] == 4) & (temp['Family Size'] == 1) & (temp['Pclass'] == 1), 'FH'] = 5
temp.loc[(temp['Gclass'] == 4) & (temp['Family Size'] == 1) & (temp['Pclass'] == 2), 'FH'] = 6
temp.loc[(temp['Gclass'] == 4) & (temp['Fgroup'] == 3), 'FH'] = 7
temp.loc[(temp['Gclass'] == 4) & (temp['Fgroup'] >= 5), 'FH'] = 8

survpct('FH')

In [ ]:
temp['MH'] = 0
temp.loc[(temp['Sex'] == 2), 'MH'] = 0
temp.loc[(temp['Gclass'] == 1), 'MH'] = 1
temp.loc[(temp['Gclass'] == 1) & (temp['Family Size'] == 2), 'MH'] = 2
temp.loc[(temp['Gclass'] == 1) & (temp['Family Size'] == 3), 'MH'] = 3
temp.loc[(temp['Gclass'] == 1) & (temp['Family Size'] == 4), 'MH'] = 4
temp.loc[(temp['Gclass'] == 1) & (temp['Family Size'] == 1) & (temp['Pclass'] == 1), 'MH'] = 5
temp.loc[(temp['Gclass'] == 1) & (temp['Family Size'] == 1) & (temp['Pclass'] == 2), 'MH'] = 6
temp.loc[(temp['Gclass'] == 1) & (temp['Fgroup'] == 3), 'MH'] = 7
temp.loc[(temp['Gclass'] == 1) & (temp['Fgroup'] >= 5), 'MH'] = 8

survpct('MH')

In [ ]:
temp['FL'] = 0
temp.loc[(temp['Gclass'] != 5), 'FL'] = 0
temp.loc[(temp['Gclass'] == 5) & (temp['Fgroup'] < 5), 'FL'] = 1
temp.loc[(temp['Gclass'] == 5) & (temp['Fgroup'] != 3), 'FL'] = 2
temp.loc[(temp['Gclass'] == 5) & (temp['FH'] == 1), 'FL'] = 3
temp.loc[(temp['Gclass'] == 5) & (temp['Family Size'] < 2), 'FL'] = 4
temp.loc[(temp['Gclass'] == 5) & (temp['Family Size'] > 4), 'FL'] = 5
temp.loc[(temp['Gclass'] == 5) & (temp['Family Size'] == 1) & (temp['Pclass'] == 3), 'FL'] = 6

In [ ]:
survpct('FL')

In [ ]:
temp['ML'] = 0
temp.loc[(temp['Gclass'] == 2) & (temp['Fgroup'] < 5), 'ML'] = 1
temp.loc[(temp['Gclass'] == 2) & (temp['Fgroup'] != 3), 'ML'] = 2
temp.loc[(temp['Gclass'] == 2) & (temp['MH'] <7), 'ML'] = 3
temp.loc[(temp['Gclass'] == 2) & (temp['Family Size'] < 2), 'ML'] = 4
temp.loc[(temp['Gclass'] == 2) & (temp['Family Size'] > 4), 'ML'] = 5
temp.loc[(temp['Gclass'] == 2) & (temp['Family Size'] == 1) & (temp['Pclass'] == 3), 'ML'] = 6
temp.loc[(temp['Gclass'] == 3) & (temp['Fgroup'] < 5), 'ML'] = 1
temp.loc[(temp['Gclass'] == 3) & (temp['Fgroup'] != 3), 'ML'] = 2
temp.loc[(temp['Gclass'] == 3) & (temp['MH'] <7), 'ML'] = 3
temp.loc[(temp['Gclass'] == 3) & (temp['Family Size'] < 2), 'ML'] = 4
temp.loc[(temp['Gclass'] == 3) & (temp['Family Size'] > 4), 'ML'] = 5
temp.loc[(temp['Gclass'] == 3) & (temp['Family Size'] == 1) & (temp['Pclass'] == 3), 'ML'] = 6

survpct('ML')

In [ ]:
from random import randint
temp['RAND'] = [ randint(1,149)  for k in temp.index]

temp.head(10)

In [ ]:
temp['Score1']= 0
temp.loc[(temp['Priority'] > 0), 'Score1'] = 1
temp.loc[(temp['RAND'] > 116), 'Score1'] = 1

temp.Score1.value_counts()

In [ ]:
from random import randint
temp['rand'] = [ randint(1,9)  for k in temp.index]

temp.head(10)

In [ ]:
temp['Score2']= 0
temp.loc[(temp['Priority'] > 0), 'Score2'] = 1
temp.loc[((temp['Family Size'] > 1) & (temp['Family Size'] < 5)), 'Score2'] = 1
temp.loc[(temp['rand'] > 7), 'Score2'] = 1

temp.Score2.value_counts()

In [ ]:
missingno.matrix(temp, figsize = (15,8))

### Deciding final features

* Now we are ready to move on.
* 이제 다음 단계로 갑니다.

* Let's make two new dataframes, one for label encoding, the other for one-hot-encoding
* 두개의 새로운 데이터 프레임을 만듭니다. 하나는 레이블 인코딩 다른 하나는 원핫 인코딩 (둘이 꼭 필요한 것이 아니라 연습이나 두 가지 방 법 모두 사용해봄)

* 대표적인 인코딩에 Label Encoding이 있는데 이는 각 항목의 값을 서열화 시켜 주~욱 줄세운 것이라 생각하시면 됩니다.
* 그 외에 자주쓰는 One Hot Encoding 같은 경우 열 내에서의 항목을 나누어서 (열이 주~욱 늘어나며) 이를 0이냐 1이냐로 구분해 놓은 것입니다.
* A typical encoding is Label Encoding, which can be considered to be a very good order of the values of each item.
* In addition, the one-of-a-kind One Hot Encoding is divided into 0 or 1 by dividing the items in the column (they increase in number).

* 다시 말씀 드려서 레이블 인코딩은 줄을 세워서 번호를 부여하는 것이고, 원핫인코딩은 긴가 아닌가 두 가지입니다.
* label encoding assign ordinal numbers while one hot encoding gives 1 or 0

* For example 16 year old is number 5 and 17 year old is number 6 ..that's the way it is for the label encoding
* 예를들어 나이별로 줄을 세워 너는 5번, 너는 6번이런식이 레이블 인코딩이고

* If you are 16 yes or no , 17 yes or no...that's one hot encoding
* 16살이야? 1, 16살 아냐 0 ..그 다음 17살이야? 1 17살 아냐 0 ..이런 식으라 열의 수가 무지하게 늘어납니다.

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

새로운 Data Frame을 만듭니다.

In [ ]:
dfl = pd.DataFrame() # for label encoding

In [ ]:
good_columns = ['Priority', 'Gclass', 'Agroup', 'Family_Survival', 'Initial','NumName', 'Initick', 'FL', 'ML', 'FH', 'MH', 'Fgroup', 'Family Size','Embarked', 'Score1', 'Score2']
dfl[good_columns] = temp[good_columns]

In [ ]:
dfl.head()

In [ ]:
dfh = dfl.copy()

In [ ]:
dfl_enc = dfl.apply(LabelEncoder().fit_transform)
                          
dfl_enc.head(20)

In [ ]:
one_hot_cols = dfh.columns.tolist()
dfh_enc = pd.get_dummies(dfh, columns=one_hot_cols)

dfh_enc.head()

# Making ML models / 머신러닝 모델 만들기

* 자, 이제 머신 러닝 모델을 만들어 보지요.
* 우선 인코딩한 파일을 train과 test로 아까 구분해 놓은 행으로 쪼갭니다

* Now let's create a machine learning model.
* First, split the encoded file into the lines that were previously separated by train and test.

In [ ]:
train = dfh_enc[:ntrain]
test = dfh_enc[ntrain:]

In [ ]:
X_test = test
X_train = train

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
ran = RandomForestClassifier(random_state=1)
knn = KNeighborsClassifier()
log = LogisticRegression()
xgb = XGBClassifier()
gbc = GradientBoostingClassifier()
svc = SVC(probability=True)
ext = ExtraTreesClassifier()
ada = AdaBoostClassifier()
gnb = GaussianNB()
gpc = GaussianProcessClassifier()
bag = BaggingClassifier()

# Prepare lists
models = [ran, knn, log, xgb, gbc, svc, ext, ada, gnb, gpc, bag]         
model_names = ['Random Forest', 'K Nearest Neighbour', 'Logistic Regression', 'XGBoost', 'Gradient Boosting', 'SVC', 'Extra Trees', 'AdaBoost', 'Gaussian Naive Bayes', 'Gaussian Process', 'Bagging Classifier']
scores = {}

# Sequentially fit and cross validate all models
for ind, mod in enumerate(models):
    mod.fit(X_train, y_train)
    acc = cross_val_score(mod, X_train, y_train, scoring = "accuracy", cv = 10)
    scores[model_names[ind]] = acc

In [ ]:
# 결과 테이블을 만듭니다.
results = pd.DataFrame(scores).T
results['mean'] = results.mean(1)

result_df = results.sort_values(by='mean', ascending=False)#.reset_index()
result_df.head(11)

In [ ]:

result_df = result_df.drop(['mean'], axis=1)
sns.boxplot(data=result_df.T, orient='h')
plt.title('Machine Learning Algorithm Accuracy Score \n')
plt.xlabel('Accuracy Score (%)');

In [ ]:
# 중요도를 보는 함수를 만듭니다.
def importance_plotting(data, xlabel, ylabel, title, n=20):
    sns.set(style="whitegrid")
    ax = data.tail(n).plot(kind='barh')
    
    ax.set(title=title, xlabel=xlabel, ylabel=ylabel)
    ax.xaxis.grid(False)
    ax.yaxis.grid(True)
    plt.show()

In [ ]:
# 데이터 프레임에 항목 중요도를 넣습니다.
fi = {'Features':train.columns.tolist(), 'Importance':xgb.feature_importances_}
importance = pd.DataFrame(fi, index=fi['Features']).sort_values('Importance', ascending=True)

In [ ]:
# 그래프 제목
title = 'Top 20 most important features in predicting survival on the Titanic: XGB'

# 그래프 그리기
importance_plotting(importance, 'Importance', 'Features', title, 20)

In [ ]:
# 중요도를 데이터프레임에 넣습니다. Logistic regression에서는 중요도보다 coefficients를 사용합니다. 
# 아래는 Features라는 열에 트레인의 열들의 이름을 리스트로 만들어서 넣고 Importance에는 Logistic regression에는 coefficient를 바꾸어 넣어라는 넘파이 명령입니다.(즉 가로를 세로로)
fi = {'Features':train.columns.tolist(), 'Importance':np.transpose(log.coef_[0])}
importance = pd.DataFrame(fi, index=fi['Features']).sort_values('Importance', ascending=True)
# 그래프 타이틀
title = 'Top 20 important features in predicting survival on the Titanic: Logistic Regression'

# 그래프 그리기
importance_plotting(importance, 'Importance', 'Features', title, 20)

In [ ]:
# 5가지 모델에 대한 항목 중요도 얻기
gbc_imp = pd.DataFrame({'Feature':train.columns, 'gbc importance':gbc.feature_importances_})
xgb_imp = pd.DataFrame({'Feature':train.columns, 'xgb importance':xgb.feature_importances_})
ran_imp = pd.DataFrame({'Feature':train.columns, 'ran importance':ran.feature_importances_})
ext_imp = pd.DataFrame({'Feature':train.columns, 'ext importance':ext.feature_importances_})
ada_imp = pd.DataFrame({'Feature':train.columns, 'ada importance':ada.feature_importances_})

# 이를 하나의 데이터프레임으로
importances = gbc_imp.merge(xgb_imp, on='Feature').merge(ran_imp, on='Feature').merge(ext_imp, on='Feature').merge(ada_imp, on='Feature')

# 항목당 평균 중요도
importances['Average'] = importances.mean(axis=1)

# 랭킹 정하기
importances = importances.sort_values(by='Average', ascending=False).reset_index(drop=True)


In [ ]:
# 중요도를 다시 데이터 프레임에 넣기
fi = {'Features':importances['Feature'], 'Importance':importances['Average']}
importance = pd.DataFrame(fi).set_index('Features').sort_values('Importance', ascending=True)

# 그래프 타이틀
title = 'Top 20 important features in predicting survival on the Titanic: 5 model average'

# 그래프 보기
importance_plotting(importance, 'Importance', 'Features', title, 20)

In [ ]:
importance1 = importance[-120:]

importance1[111:120]

In [ ]:
# 영양가 있는 120개만 넣기
mylist = list(importance1.index)

In [ ]:
train1 = pd.DataFrame()
test1 = pd.DataFrame()

for i in mylist:
    train1[i] = train[i]
    test1[i]= test[i]
    
train1.head()


In [ ]:
train = train1
test = test1

# 모델의 변수를 다시 정의하고
X_train = train
X_test = test

# 바꿉니다.
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
ran = RandomForestClassifier(random_state=1)
knn = KNeighborsClassifier()
log = LogisticRegression()
xgb = XGBClassifier(random_state=1)
gbc = GradientBoostingClassifier(random_state=1)
svc = SVC(probability=True)
ext = ExtraTreesClassifier(random_state=1)
ada = AdaBoostClassifier(random_state=1)
gnb = GaussianNB()
gpc = GaussianProcessClassifier()
bag = BaggingClassifier(random_state=1)

# Prepare lists
models = [ran, knn, log, xgb, gbc, svc, ext, ada, gnb, gpc, bag]         
model_names = ['Random Forest', 'K Nearest Neighbour', 'Logistic Regression', 'XGBoost', 'Gradient Boosting', 'SVC', 'Extra Trees', 'AdaBoost', 'Gaussian Naive Bayes', 'Gaussian Process', 'Bagging Classifier']
scores2 = {}

# Sequentially fit and cross validate all models
for ind, mod in enumerate(models):
    mod.fit(X_train, y_train)
    acc = cross_val_score(mod, X_train, y_train, scoring = "accuracy", cv = 10)
    scores2[model_names[ind]] = acc

In [ ]:
# 결과 테이블을 만듭니다.
results = pd.DataFrame(scores2).T
results['mean'] = results.mean(1)

result_df = results.sort_values(by='mean', ascending=False)#.reset_index()
result_df.head(11)
result_df = result_df.drop(['mean'], axis=1)
sns.boxplot(data=result_df.T, orient='h')
plt.title('Machine Learning Algorithm Accuracy Score \n')
plt.xlabel('Accuracy Score (%)');

## 하이퍼파라미터 튜닝

### SVC
* Scikit-Learn에서는 3가지 모형 최적화 도구를 지원하는데 validation_curve/ GridSearchCV/ ParameterGrid이다
* fit 메소드를 호출하면 grid search가 자동으로 여러개의 내부 모형을 생성하고 이를 모두 실행시켜서 최적 파라미터를 찾는다.

* bestscore는 최고 점수이고 best estimator는 최고 점수를 낸 파라미터를 가진 모형
* c값과 gamma값은 10의 배수로 일반적으로 한다.
* 감마 매개 변수는 단일 학습 예제의 영향이 도달하는 정도를 정의하며 낮은 값은 'far'를, 높은 값은 'close'를 나타냅니다. 감마 매개 변수는 서포트 벡터로 모델에 의해 선택된 샘플의 영향 반경의 역으로 볼 수 있습니다.
* C 매개 변수는 의사 결정 표면의 단순성에 대한 훈련 예제의 오 분류를 제거합니다. C가 낮을수록 결정 표면이 매끄럽고 높은 C는 모델이 더 많은 샘플을 서포트 벡터로 자유롭게 선택할 수 있도록하여 모든 학습 예제를 올바르게 분류하는 것을 목표로합니다.
* Verbose는 불리안 값으로 True로 넣으면 꼬치 꼬치 다 알려주는데, 대신 시간이 좀 더 오래 걸립니다.
* cv =5는 5 fold로 교차 검증한다는 뜻입니다.

In [ ]:
# 파라미터 서치
Cs = [0.01, 0.1, 1, 5, 10, 15, 20, 50]
gammas = [0.001, 0.01, 0.1]

# 파라미터 그리드 셋팅
hyperparams = {'C': Cs, 'gamma' : gammas}

# 교차검증
gd=GridSearchCV(estimator = SVC(probability=True), param_grid = hyperparams, 
                verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

# 모델 fiting 및 결과
gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### Gradient Boosting Classifier
* learning_rate는 각 트리의 기여를 줄이는 역할을 합니다.
* n_estimator는 각 경우의 트리 숫자입니다.

In [ ]:
learning_rate = [0.01, 0.05, 0.1, 0.2, 0.5]
n_estimators = [100, 1000, 2000]
max_depth = [3, 5, 10, 15]

hyperparams = {'learning_rate': learning_rate, 'n_estimators': n_estimators}

gd=GridSearchCV(estimator = GradientBoostingClassifier(), param_grid = hyperparams, 
                verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### Logistic Regression
* Penalty - L1 을 사용하는 회귀 모델을 Lasso Regression이라고하고 L2를 사용하는 모델을 Ridge Regression이라고합니다. 이 둘의 주요 차이점은 페널티입니다. 릿지 회귀는 손실 함수에 페널티 항으로 계수의 "제곱 크기"를 추가합니다. L2-norm이 오차를 제곱하기 때문에 (오류> 1 인 경우 로트가 증가 함) 모델은 L1-norm보다 훨씬 큰 오차 (e vs e ^ 2)를 보게되므로 훨씬 더 민감합니다. 따라서 오류를 최소화하기 위해 모델을 조정해줍니다.
* C는 estimator 입니다. logspace 1차원 10개 배열로 0에서 4까지를 estimator로 놓은 것입니다.

In [ ]:
penalty = ['l1', 'l2']
C = np.logspace(0, 4, 10)

hyperparams = {'penalty': penalty, 'C': C}

gd=GridSearchCV(estimator = LogisticRegression(), param_grid = hyperparams, 
                verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### XGBoost Step 1.

In [ ]:
learning_rate = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2]
n_estimators = [10, 50, 100, 250, 500, 1000]

hyperparams = {'learning_rate': learning_rate, 'n_estimators': n_estimators}

gd=GridSearchCV(estimator = XGBClassifier(), param_grid = hyperparams, 
                verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### XGB Step 2.

In [ ]:
max_depth = [3, 4, 5, 6, 7, 8, 9, 10]
min_child_weight = [1, 2, 3, 4, 5, 6]

hyperparams = {'max_depth': max_depth, 'min_child_weight': min_child_weight}

gd=GridSearchCV(estimator = XGBClassifier(learning_rate=0.2, n_estimators=10), param_grid = hyperparams, 
                verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### XGB Step 3.


In [ ]:
gamma = [i*0.1 for i in range(0,5)]

hyperparams = {'gamma': gamma}

gd=GridSearchCV(estimator = XGBClassifier(learning_rate=0.2, n_estimators=10, max_depth=6, 
                                          min_child_weight=1), param_grid = hyperparams, 
                verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### XGB Step 4

In [ ]:
subsample = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
colsample_bytree = [0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    
hyperparams = {'subsample': subsample, 'colsample_bytree': colsample_bytree}

gd=GridSearchCV(estimator = XGBClassifier(learning_rate=0.2, n_estimators=10, max_depth=6, 
                                          min_child_weight=1, gamma=0), param_grid = hyperparams, 
                verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### XGB Step 5

In [ ]:

reg_alpha = [1e-5, 1e-2, 0.1, 1, 100]
    
hyperparams = {'reg_alpha': reg_alpha}

gd=GridSearchCV(estimator = XGBClassifier(learning_rate=0.2, n_estimators=10, max_depth=6, 
                                          min_child_weight=1, gamma=0, subsample=1, colsample_bytree=1),
                                         param_grid = hyperparams, verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### Gaussian Process

In [ ]:
n_restarts_optimizer = [0, 1, 2, 3]
max_iter_predict = [1, 2, 5, 10, 20, 35, 50, 100]
warm_start = [True, False]

hyperparams = {'n_restarts_optimizer': n_restarts_optimizer, 'max_iter_predict': max_iter_predict, 'warm_start': warm_start}

gd=GridSearchCV(estimator = GaussianProcessClassifier(), param_grid = hyperparams, 
                verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### Adaboost.

In [ ]:
n_estimators = [10, 100, 200, 500]
learning_rate = [0.001, 0.01, 0.1, 0.5, 1, 1.5, 2]

hyperparams = {'n_estimators': n_estimators, 'learning_rate': learning_rate}

gd=GridSearchCV(estimator = AdaBoostClassifier(), param_grid = hyperparams, 
                verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### KNN

In [ ]:
n_neighbors = [1, 2, 3, 4, 5]
algorithm = ['auto']
weights = ['uniform', 'distance']
leaf_size = [1, 2, 3, 4, 5, 10]

hyperparams = {'algorithm': algorithm, 'weights': weights, 'leaf_size': leaf_size, 
               'n_neighbors': n_neighbors}

gd=GridSearchCV(estimator = KNeighborsClassifier(), param_grid = hyperparams, 
                verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

# Fitting model and return results
gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### Random Forest.

In [ ]:
n_estimators = [10, 50, 100, 200]
max_depth = [3, None]
max_features = [0.1, 0.2, 0.5, 0.8]
min_samples_split = [2, 6]
min_samples_leaf = [2, 6]

hyperparams = {'n_estimators': n_estimators, 'max_depth': max_depth, 'max_features': max_features,
               'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}

gd=GridSearchCV(estimator = RandomForestClassifier(), param_grid = hyperparams, 
                verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### Extra Trees

In [ ]:
n_estimators = [10, 25, 50, 75, 100]
max_depth = [3, None]
max_features = [0.1, 0.2, 0.5, 0.8]
min_samples_split = [2, 10]
min_samples_leaf = [2, 10]

hyperparams = {'n_estimators': n_estimators, 'max_depth': max_depth, 'max_features': max_features,
               'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}

gd=GridSearchCV(estimator = ExtraTreesClassifier(), param_grid = hyperparams, 
                verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### Bagging Classifier

In [ ]:
n_estimators = [10, 50, 75, 100, 200]
max_samples = [0.1, 0.2, 0.5, 0.8, 1.0]
max_features = [0.1, 0.2, 0.5, 0.8, 1.0]

hyperparams = {'n_estimators': n_estimators, 'max_samples': max_samples, 'max_features': max_features}

gd=GridSearchCV(estimator = BaggingClassifier(), param_grid = hyperparams, 
                verbose=True, cv=5, scoring = "accuracy", n_jobs=-1)

gd.fit(X_train, y_train)
print(gd.best_score_)
print(gd.best_params_)

### 모델 재 트레이닝

In [ ]:

# 튜닝 모델 시작
# sample을 split하는 것은 전체데이터 80%를 트레인셋에 20%는 테스트셋에 줌  
ran = RandomForestClassifier(max_depth=3, max_features=0.8, min_samples_leaf=6, min_samples_split=2, n_estimators=200, random_state=1)

knn = KNeighborsClassifier(leaf_size=1, n_neighbors=2, weights='uniform')

log = LogisticRegression(C=1.0, penalty='l2')

xgb = XGBClassifier(learning_rate=0.05, n_estimators=250, max_depth=3, 
                                          min_child_weight=6, gamma=0.4, subsample=1, colsample_bytree=0.65, reg_alpha=1e-05)

gbc = GradientBoostingClassifier(learning_rate=0.05, max_depth=3, n_estimators=100)

svc = SVC(probability=True, gamma=0.001, C=10)

ext = ExtraTreesClassifier(max_depth=3, max_features=0.08, min_samples_leaf=6, min_samples_split=2, n_estimators=200, random_state=1)

ada = AdaBoostClassifier(learning_rate=1, n_estimators=200, random_state=1)

gpc = GaussianProcessClassifier(max_iter_predict=1, n_restarts_optimizer=0, warm_start=True)

bag = BaggingClassifier(max_features=0.2, max_samples=0.2, n_estimators=100, random_state=1)

# 리스트
models = [ran, knn, log, xgb, gbc, svc, ext, ada, gnb, gpc, bag]         
model_names = ['Random Forest', 'K Nearest Neighbour', 'Logistic Regression', 'XGBoost', 'Gradient Boosting', 'SVC', 'Extra Trees', 'AdaBoost', 'Gaussian Naive Bayes', 'Gaussian Process', 'Bagging Classifier']
scores3 = {}

# Sequentially fit and cross validate all models
for ind, mod in enumerate(models):
    mod.fit(X_train, y_train)
    acc = cross_val_score(mod, X_train, y_train, scoring = "accuracy", cv = 10)
    scores3[model_names[ind]] = acc

In [ ]:
results = pd.DataFrame(scores).T
results['mean'] = results.mean(1)
result_df = results.sort_values(by='mean', ascending=False)
result_df.head(11)


result_df = result_df.drop(['mean'], axis=1)
sns.boxplot(data=result_df.T, orient='h')
plt.title('Machine Learning Algorithm Accuracy Score \n')
plt.xlabel('Accuracy Score (%)');

## Voting (Hard/Soft)

In [ ]:
#튜닝한 파라미터로 하드보팅
grid_hard = VotingClassifier(estimators = [('Random Forest', ran), 
                                           ('Logistic Regression', log),
                                           ('XGBoost', xgb),
                                           ('Gradient Boosting', gbc),
                                           ('Extra Trees', ext),
                                           ('AdaBoost', ada),
                                           ('Gaussian Process', gpc),
                                           ('SVC', svc),
                                           ('K Nearest Neighbour', knn),
                                           ('Bagging Classifier', bag)], voting = 'hard')

grid_hard_cv = model_selection.cross_validate(grid_hard, X_train, y_train, cv=10)
grid_hard.fit(X_train, y_train)

print("Hard voting on test set score mean: {:.2f}". format(grid_hard_cv['test_score'].mean() * 100))

In [ ]:
grid_soft = VotingClassifier(estimators = [('Random Forest', ran), 
                                           ('Logistic Regression', log),
                                           ('XGBoost', xgb),
                                           ('Gradient Boosting', gbc),
                                           ('Extra Trees', ext),
                                           ('AdaBoost', ada),
                                           ('Gaussian Process', gpc),
                                           ('SVC', svc),
                                           ('K Nearest Neighbour', knn),
                                           ('Bagging Classifier', bag)], voting = 'soft')

grid_soft_cv = model_selection.cross_validate(grid_soft, X_train, y_train, cv=10)
grid_soft.fit(X_train, y_train)

print("Soft voting on test set score mean: {:.2f}". format(grid_soft_cv['test_score'].mean() * 100))

## 마지막 모델 예측

In [ ]:
# Final predictions

predictions = grid_hard.predict(X_test)

submission = pd.concat([pd.DataFrame(passId), pd.DataFrame(predictions)], axis = 'columns')

submission.columns = ["PassengerId", "Survived"]
submission.to_csv('titanic_submission305.csv', header = True, index = False)

In [ ]:
# Final predictions2

predictions = grid_soft.predict(X_test)

submission = pd.concat([pd.DataFrame(passId), pd.DataFrame(predictions)], axis = 'columns')

submission.columns = ["PassengerId", "Survived"] 
submission.to_csv('titanic_submission306.csv', header = True, index = False)

### 제출

In [ ]:
# And we finally make a submission 그리고 제출 합니다.
# Please make sure you "commit" (It take a few minutes) / commit버턴을 누르시는 것을 잊지 마세요 (몇 분 걸립니다)
# And then you will see the submission file on the top right hand side at Data>Output>Kaggle/working / 그럼 우측 상단 데이터 아웃풋에서 제출용 결과물이 나올 것입니다.